# Offshore Wind Farming

## Objective and Prerequisites

In this example, we will solve the problem of how to minimize the cost of laying underwater cables to collect electricity produced by an offshore wind farm. We will construct a mixed-integer programming (MIP)  model of this problem, implement this model in the Gurobi Python interface, and compute an optimal solution.

This modeling example is at the beginner level, where we assume that you know Python and that you have some knowledge about building mathematical optimization models.

**Note:** You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an [evaluation license](https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=Github&utm_medium=website_JupyterME&utm_campaign=CommercialDataScience) as a *commercial user*, or download a [free license](https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=Github&utm_medium=website_JupyterME&utm_campaign=AcademicDataScience) as an *academic user*.

## Motivation

Global climate change has already had observable effects on the environment. Glaciers have shrunk, ice on rivers and lakes is breaking up earlier than expected, plant and animal species have  been affected and trees are flowering sooner than expected. The potential future effects of global climate change include more frequent wildfires, longer periods of drought in some regions and an increase in the number, duration and intensity of tropical storms. [1]

Climate change mitigation consists of actions to limit the magnitude or rate of global warming and its related effects.
The first challenge for climate change mitigation is eliminating the burning of coal, oil and, eventually, natural gas. This is perhaps the most daunting challenge as denizens of richer nations literally eat, wear, work, play and even sleep on the products made from fossil fuels. Also, citizens of developing nations want and arguably deserve the same comforts. There are no perfect solutions for reducing dependence on fossil fuels (for example, carbon neutral biofuels can drive up the price of food and lead to forest destruction, and while nuclear power does not emit greenhouse gases, it does produce radioactive waste). Other alternatives include plant-derived plastics, biodiesel, and wind power. [2]

Offshore wind power is the use of wind farms constructed in bodies of water, usually in the ocean, to harvest wind energy to generate electricity. Higher wind speeds are available offshore compared to on land, so offshore wind power’s electricity generation is higher per amount of capacity installed. 

The advantage of locating wind turbines offshore is that the wind is much stronger off the coasts, and unlike wind over the continent, offshore breezes can be strong in the afternoon, matching the time when people are using the most electricity. Offshore turbines can also be located close to the load centers along the coasts, such as large cities, eliminating the need for new long-distance transmission lines.


## Problem Description

An offshore wind farm is a collection of wind turbines placed at sea to take advantage of the strong offshore winds. These strong winds produce more electricity, but offshore wind farms are more expensive to install and operate than those on land.

We will use a MIP model to reduce part of the cost of building an offshore wind farm. We will compute a plan for how to lay the underwater cables that connect the turbines. These cables are necessary to transfer the power produced by the turbines to land. The plan we compute will minimize the cost to install the underwater cables, while ensuring that each turbine is connected to the shore and each cable has sufficient capacity to handle the electrical current generated.

In our example, a wind farm is being built off the west coast of Denmark. There is a power station on the coast where all the electricity must be transferred to be distributed to the electric grid. There are also transfer stations in the wind farm where the power from several turbines can be collected and transferred along a single cable to the shore.

There are two factors we must consider when installing the cables. First, there is a fixed cost to lay a cable on the sea floor. This cost is proportional to the distance between the two stations the cable connects. Second, we must consider how much current will flow through the cables. Connections that carry large currents need thick cables. Thick cables are more expensive than thin cables.

The goal of this optimization problem is to decide which cables should be laid to connect the wind farm power network at a minimum cost.

The model of offshore wind farming optimization problem is an instance of a more general optimization model known as fixed charge network flow problem. Fixed charge network flow problems can be applied to a large number of business problems -for example, in the planning of communication and transport networks.



## Solution Approach

Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex decision problem. The Gurobi Optimizer solves such models using state-of-the-art mathematics and computer science.

A mathematical optimization model has five components, namely:

* Sets and indices.
* Parameters.
* Decision variables.
* Objective function(s).
* Constraints.

We now present a MIP formulation for the offshore wind farming problem.

## Model Formulation

### Sets and Indices

$G(V,E)$: Graph that represents the wind farm network, where $V$ is the set of vertices and $E$ is the set of edges. The turbines, transfer stations, and power stations are vertices in the set of vertices $V$ of the graph. The set of potential cables are the edges of the graph. 

### Parameters

$s_{i} \in \mathbb{R}$: Power supplied at vertex $i \in V$. Since turbines supply power,  they are source vertices with $s_{i} > 0$. Transfer stations do not supply or remove power from the network so they have $s_{i} = 0$. The power station on the coast is a sink that remove all power from the wind farm so it has $s_{i} < 0$.

$u_{i,j} \in \mathbb{R}^+ $: Maximum current capacity a cable can handle from vertex $i \in V$ to vertex $j \in V$.

$c_{i,j} \in \mathbb{R}^+$: Cost per unit of current flow going from vertex $i \in V$ to vertex $j \in V$, i.e. the price we must pay to increase the thickness of the cable to handle an increase in current. 

$f_{i,j} \in \mathbb{R}^+$: Fixed cost of laying a cable from vertex $i \in V$ to vertex $j \in V$, and is the result of multiplying the distance between vertices by the cost per mile.

### Decision Variables

$install_{i,j} \in \{0, 1 \}$: This variable is equal to 1 if we lay a cable from vertex $i \in V$ to vertex $j \in V$; and 0 otherwise.

$flow_{i,j} \geq 0$: This non-negative continuous variable represents the amount of current flowing from vertex $i \in V$ to vertex $j \in V$.

The goal of this optimization model is to decide which of these potential edges in the graph should be used at a minimum cost.

### Objective Function

- **Total costs**. We want to minimize the total cost to install the cables. The term on the left is the variable costs (i.e. those that vary according to the current in the cable). The term on right is the fixed cost to install the cable.

\begin{equation}
\text{Max} \quad Z = \sum_{(i,j) \in E} c_{i,j} \cdot flow_{i,j} + \sum_{(i,j) \in E} f_{i,j} \cdot install_{i,j} 
\tag{0}
\end{equation}

### Constraints

- **Balance**. For each vertex $i \in V$, we want to ensure the conservation of current in the network.

\begin{equation}
\sum_{j:(i,j) \in E} flow_{i,j} - \sum_{j:(j,i) \in E} flow_{j,i} = s_{i} \quad \forall i \in V
\tag{1}
\end{equation}

- **Capacity**. For each edge $(i,j) \in E$, we want to enforce the limits on the maximum current capacity of each cable.

\begin{equation}
0 \leq flow_{i,j} \leq u_{i,j} \cdot install_{i,j}  \quad \forall (i,j) \in E
\tag{2}
\end{equation}

## Python Implementation

This example considers three turbines, one transfer station, and two power stations. The current flowing out at each vertex of the wind farm network is presented in the following table. Recall that since turbines supply power their capacity is positive. Transfer stations do not supply or remove power from the network so they have a capacity of zero.  The power stations on the coast are sinks that remove all power from the wind farm network so they have demand of power, in this case we use a negative number.

| <i></i> | Capacity in MW |  
| --- | --- | 
| vertex 1 | 4 |
| vertex 2 | 3 |
| vertex 3 | 2 |
| vertex 4 | 0 |
| vertex 5 | -6 |
| vertex 6 | -3 |


The capacity, flow cost, and fixed cost of each edge in the wind farm network are provided in the following table.

| <i></i> | Capacity in MW  | Flow cost in millions of Euros | Fixed cost in millions of Euros| 
| --- | --- | --- | --- |
| Edge: (0,4) | 4 | 1 | 1 |
| Edge: (0,3) | 2 | 1 | 1 |
| Edge: (1,3) | 3 | 1 | 1 |
| Edge: (2,5) | 2 | 1 | 1 |
| Edge: (3,4) | 2 | 1 | 1 |
| Edge: (3,5) | 1 | 1 | 1 |


We now import the Gurobi Python Module. Then, we initialize the data structures with the given data.

In [1]:
import gurobipy as gp
from gurobipy import GRB

# Parameters

vertices = {0: 4, 1: 3, 2: 2, 3: 0, 4: -6, 5: -3}
edges, cap, flow_cost, fixed_cost = gp.multidict({
    (0,4): [4,1,1],
    (0,3): [2,1,1],
    (1,3): [3,1,1],
    (2,5): [2,1,1],
    (3,4): [2,1,1],
    (3,5): [1,1,1]
})

### Model Deployment

We now determine the MIP model for the offshore wind farming problem, by defining the decision variables, constraints, and objective function. Next, we start the optimization process and Gurobi finds the plan to lay cables at the offshore wind farming network that minimizes total costs.

In [2]:
# MIP  model formulation

m = gp.Model("offshore_wind_farming")

# Add variables
install = m.addVars(edges, vtype=GRB.BINARY, name="Install")
flow = m.addVars(edges, vtype=GRB.CONTINUOUS, name="Flow")

# Add constraints
m.addConstrs((flow.sum(v,'*') - flow.sum('*',v) == supply for v, supply in vertices.items()), name="Flow_conservation")
m.addConstrs((flow[e] <= cap[e]*install[e] for e in edges), name="Install2flow")

# Set objective
m.setObjective(flow.prod(flow_cost) + install.prod(fixed_cost), GRB.MINIMIZE)

m.optimize()

Using license file c:\gurobi\gurobi.lic
Set parameter TokenServer to value SANTOS-SURFACE-
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 12 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x03b3989f
Variable types: 6 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 6e+00]
Presolve removed 12 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 17 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.700000000000e+01, best bound 1.700000000000e+01, gap 0.0000%


## Analysis


The result of the optimization model shows that the minimum total cost value is 17 million Euros. Let's see the solution that achieves that optimal result.

### Cable Installation Plan

This plan determines the layout of cables in the offshore wind farming network.

In [3]:
# display which edges in the offshore wind farming network we plan to install.

for origin, end in install.keys():
    if (abs(install[origin, end].x) > 0.5):
        print(f"Install cable from location {origin + 1} to location {end + 1} in the offshore wind farming network ")
        


Install cable from location 1 to location 5 in the offshore wind farming network 
Install cable from location 2 to location 4 in the offshore wind farming network 
Install cable from location 3 to location 6 in the offshore wind farming network 
Install cable from location 4 to location 5 in the offshore wind farming network 
Install cable from location 4 to location 6 in the offshore wind farming network 


### Cable Capacity Plan 

This plan determines the current flow capacity in MW of each cable installed.

In [4]:
# Current flow capacity of each cable installed

for origin, end in flow.keys():
    if (abs(flow[origin, end].x) > 1e-6):
         print(f"The capacity of cable installed from location {origin + 1} to location {end + 1} is {flow[origin, end].x} MW ")

The capacity of cable installed from location 1 to location 5 is 4.0 MW 
The capacity of cable installed from location 2 to location 4 is 3.0 MW 
The capacity of cable installed from location 3 to location 6 is 2.0 MW 
The capacity of cable installed from location 4 to location 5 is 2.0 MW 
The capacity of cable installed from location 4 to location 6 is 1.0 MW 


##  Conclusion

In this example, we addressed an offshore wind farming problem where we want to minimize the cost of laying underwater cables to collect electricity produced by an offshore wind farm network. We learned how to formulate the problem as a MIP model. Also, we learned how to implement the MIP model formulation and solve it using the Gurobi Python API.

## References

[1] https://climate.nasa.gov/effects/

[2] https://www.scientificamerican.com/article/10-solutions-for-climate-change/

Copyright © 2020 Gurobi Optimization, LLC